<h1>Decision Tree</h1>

In [1]:
library(mlr)
source('../utils.r')

set.seed(42)

folder_name = '../../raw_data' 
file_name   = 'german.csv'

Loading required package: ParamHelpers
Warning message:
“replacing previous import ‘BBmisc::isFALSE’ by ‘backports::isFALSE’ when loading ‘mlr’”

<h2>1. Dataprep</h2>

In [2]:
data = read.csv(file=sprintf('%s/%s',folder_name,file_name))

In [3]:
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))
head(data)

NRow: 1000
NCol: 22

X,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21
1,A11,6,A34,A43,1169,A65,A75,4,A93,⋯,A121,67,A143,A152,2,A173,good,A192,A201,1
2,A12,48,A32,A43,5951,A61,A73,2,A92,⋯,A121,22,A143,A152,1,A173,good,A191,A201,2
3,A14,12,A34,A46,2096,A61,A74,2,A93,⋯,A121,49,A143,A152,1,A172,bad,A191,A201,1
4,A11,42,A32,A42,7882,A61,A74,2,A93,⋯,A122,45,A143,A153,1,A173,bad,A191,A201,1
5,A11,24,A33,A40,4870,A61,A73,3,A93,⋯,A124,53,A143,A153,2,A173,bad,A191,A201,2
6,A14,36,A32,A46,9055,A65,A73,2,A93,⋯,A124,35,A143,A153,1,A172,bad,A192,A201,1


<p style='color: red'> ATENTION: </p>Specifically in R, in classification problem, target must be set as Factor.

In [4]:
data$V21 = as.factor(data$V21)

MLR works only with features and target, this means that others columns must be dorped.

In [5]:
drops = c('X')
data  = data[ , !(names(data) %in% drops)]
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))

NRow: 1000
NCol: 21

<h2>2. Modeling</h2>

Function
```R
makeLearner(cl, id = cl, predict.type = "response", predict.threshold = NULL, 
            fix.factors.prediction = FALSE, ..., par.vals = list(), config = list())
```
Param.:

* cl: [character(1)] Class of learner. By convention, all classification learners start with “classif.”. A list of all integrated learners is available on the learners help page < https://mlr-org.github.io/mlr-tutorial/release/html/integrated_learners/ >.
* predict: [character(1)] “response” (= labels) or “prob” (= probabilities and labels by selecting the ones with maximal probability). Default is “response”.
* par.vals: [list] Optional list of named (hyper)parameters. The arguments in ... take precedence over values in this list. We strongly encourage you to use one or the other to pass (hyper)parameters to the learner but not both.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner <br>
Rpart Parameters: https://www.rdocumentation.org/packages/rpart/versions/4.1-11/topics/rpart.control

In [6]:
cl = "classif.rpart"

List all parameters that can be used in this classifier.
The value must be set in par.vals parameter.

In [7]:
getParamSet(cl)

                   Type len  Def   Constr Req Tunable Trafo
minsplit        integer   -   20 1 to Inf   -    TRUE     -
minbucket       integer   -    - 1 to Inf   -    TRUE     -
cp              numeric   - 0.01   0 to 1   -    TRUE     -
maxcompete      integer   -    4 0 to Inf   -    TRUE     -
maxsurrogate    integer   -    5 0 to Inf   -    TRUE     -
usesurrogate   discrete   -    2    0,1,2   -    TRUE     -
surrogatestyle discrete   -    0      0,1   -    TRUE     -
maxdepth        integer   -   30  1 to 30   -    TRUE     -
xval            integer   -   10 0 to Inf   -   FALSE     -
parms           untyped   -    -        -   -    TRUE     -

In [8]:
learner = makeLearner(cl = cl
                     , predict.type = "prob"
                     , par.vals = list(minsplit = 20
                                      ,cp = 0.01 
                                      ,parms = list(split = 'information') #gini or information
                                      )
                     )

Function
```R
makeClassifTask(id = deparse(substitute(data)), data, target, weights = NULL, blocking = NULL, 
                positive = NA_character_, fixup.data = "warn", check.data = TRUE)
```
Param.:

* data: [data.frame] A data frame containing the features and target variable(s).
* target: [character(1)] Name of the target variable.
* positive: [character(1)] Positive class for binary classification (otherwise ignored and set to NA). Default is the first factor level of the target attribute.
* fixup.data: [character(1)] Should some basic cleaning up of data be performed? Currently this means removing empty factor levels for the columns. Possible coices are: “no” = Don't do it. “warn” = Do it but warn about it. “quiet” = Do it but keep silent. Default is “warn”.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner

In [9]:
task = makeClassifTask( data = data
                      , target = 'V21'
                      , positive = '2'
                      , fixup.data = 'no'
                      )

Function:
```R
makeResampleDesc(method, predict = "test", ..., stratify = FALSE, stratify.cols = NULL)
```
Param.:

* method: [character(1)] “CV” for cross-validation, “LOO” for leave-one-out, “RepCV” for repeated cross-validation, “Bootstrap” for out-of-bag bootstrap, “Subsample” for subsampling, “Holdout” for holdout.
* predict: What to predict during resampling: “train”, “test” or “both” sets. Default is “test”.
* ... : [any] Further parameters for strategies.
    * iters [integer(1)] Number of iterations, for “CV”, “Subsample” and “Boostrap”.
    * split [numeric(1)] Proportion of training cases for “Holdout” and “Subsample” between 0 and 1. Default is 2/3.
    * reps [integer(1)] Repeats for “RepCV”. Here iters = folds * reps. Default is 10.
    * folds [integer(1)] Folds in the repeated CV for RepCV. Here iters = folds * reps. Default is 10.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeResampleDesc

In [10]:
resample = makeResampleDesc( method = "CV"
                           , iters = 10
                           , predict = 'both'
                           , stratify = FALSE
                           )

List of performance measures:

Doc.: http://mlr-org.github.io/mlr-tutorial/release/html/measures/

In [11]:
measures = list(mmce #MMCE 
               ,acc  #acuracia
               ,f1   #f1
               ,ppv  #precision
               ,tpr  #recall
               ,auc  #AUC
               ,gini #Gini
               ,timetrain #tempo execucao
               )

Function:
```R
resample(learner, task, resampling, measures, weights = NULL, models = FALSE, extract, 
         keep.pred = TRUE, ..., show.info = getMlrOption("show.info"))
```
Param.:

* learner: [Learner] The learner.
* task: [Task] The task.
* resampling: [ResampleInstance] Resampling strategy.
* measures: [Measure | list of Measure] Performance measure(s) to evaluate. Default is mean misclassification error (mmce)
* weights: [numeric] Optional, non-negative case weight vector to be used during fitting. If given, must be of same length as observations in task and in corresponding order. Overwrites weights specified in the task. By default NULL which means no weights are used unless specified in the task.
* models: [logical(1)] Should all fitted models be returned? Default is FALSE.
* keep.pred: [logical(1)] Keep the prediction data in the pred slot of the result object. If you do many experiments (on larger data sets) these objects might unnecessarily increase object size / mem usage, if you do not really need them. In this case you can set this argument to FALSE. Default is TRUE.
* show.info: [logical(1)] Print verbose output on console? Default is set via configureMlr.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/resample

In [12]:
r = resample(learner = learner
            ,task = task 
            ,resampling = resample 
            ,measures = measures
            #---------------------#
            ,models = TRUE
            ,keep.pred = FALSE
            ,show.info = TRUE
            )

[Resample] cross-validation iter 1: mmce.test.mean=0.25,acc.test.mean=0.75,f1.test.mean=0.468,ppv.test.mean=0.55,tpr.test.mean=0.407,auc.test.mean=0.681,gini.test.mean=0.362,timetrain.test.mean=0.025
[Resample] cross-validation iter 2: mmce.test.mean=0.31,acc.test.mean=0.69,f1.test.mean=0.436,ppv.test.mean=0.545,tpr.test.mean=0.364,auc.test.mean=0.683,gini.test.mean=0.367,timetrain.test.mean=0.016
[Resample] cross-validation iter 3: mmce.test.mean=0.22,acc.test.mean=0.78,f1.test.mean=0.593,ppv.test.mean=0.762,tpr.test.mean=0.485,auc.test.mean=0.687,gini.test.mean=0.374,timetrain.test.mean=0.019
[Resample] cross-validation iter 4: mmce.test.mean=0.29,acc.test.mean=0.71,f1.test.mean=0.473,ppv.test.mean=0.565,tpr.test.mean=0.406,auc.test.mean=0.742,gini.test.mean=0.484,timetrain.test.mean=0.016
[Resample] cross-validation iter 5: mmce.test.mean=0.26,acc.test.mean=0.74,f1.test.mean=0.458,ppv.test.mean=0.611,tpr.test.mean=0.367,auc.test.mean=0.698,gini.test.mean=0.396,timetrain.test.mean=0.

<h2>3. Result Analysis</h2>

Train Measures

In [13]:
r$measures.train

iter,mmce,acc,f1,ppv,tpr,auc,gini,timetrain
1,0.1977778,0.8022222,0.6468254,0.7056277,0.5970696,0.7858107,0.5716214,0.025
2,0.1966667,0.8033333,0.6273684,0.7163462,0.5580524,0.7888274,0.5776547,0.016
3,0.1700000,0.8300000,0.6858316,0.7590909,0.6254682,0.8188905,0.6377810,0.019
4,0.1977778,0.8022222,0.6454183,0.6923077,0.6044776,0.7832278,0.5664557,0.016
5,0.2033333,0.7966667,0.5632458,0.7919463,0.4370370,0.7739536,0.5479071,0.014
6,0.1888889,0.8111111,0.6382979,0.7462687,0.5576208,0.7899982,0.5799963,0.013
7,0.1877778,0.8122222,0.6167800,0.7816092,0.5093633,0.7769642,0.5539284,0.013
8,0.2055556,0.7944444,0.6232179,0.7018349,0.5604396,0.7770183,0.5540366,0.014
9,0.1966667,0.8033333,0.6177106,0.7606383,0.5200000,0.7808553,0.5617105,0.014
10,0.1966667,0.8033333,0.5986395,0.7764706,0.4870849,0.7724291,0.5448583,0.012


Test Measures

In [14]:
r$measures.test

iter,mmce,acc,f1,ppv,tpr,auc,gini,timetrain
1,0.25,0.75,0.4680851,0.5500000,0.4074074,0.6811263,0.3622527,0.025
2,0.31,0.69,0.4363636,0.5454545,0.3636364,0.6834012,0.3668024,0.016
3,0.22,0.78,0.5925926,0.7619048,0.4848485,0.6872456,0.3744912,0.019
4,0.29,0.71,0.4727273,0.5652174,0.4062500,0.7419577,0.4839154,0.016
5,0.26,0.74,0.4583333,0.6111111,0.3666667,0.6980952,0.3961905,0.014
6,0.27,0.73,0.5090909,0.5833333,0.4516129,0.7218326,0.4436653,0.013
7,0.22,0.78,0.6071429,0.7391304,0.5151515,0.8066486,0.6132972,0.013
8,0.24,0.76,0.5384615,0.5600000,0.5185185,0.7085236,0.4170472,0.014
9,0.20,0.80,0.5238095,0.6470588,0.4400000,0.7146667,0.4293333,0.014
10,0.26,0.74,0.4347826,0.5882353,0.3448276,0.7141816,0.4283633,0.012


Train Aggregated Result

In [15]:
apply(r$measures.train,2,mean)

iter      mmce       acc        f1       ppv       tpr       auc      gini 
5.5000000 0.1941111 0.8058889 0.6263335 0.7432140 0.5456613 0.7847975 0.5695950 
timetrain 
0.0156000

Test Aggregated Result

In [16]:
apply(r$measures.test,2,mean)

iter      mmce       acc        f1       ppv       tpr       auc      gini 
5.5000000 0.2520000 0.7480000 0.5041389 0.6151446 0.4298919 0.7157679 0.4315358 
timetrain 
0.0156000

Run Time in seconds

In [17]:
r$runtime

[1] 0.6270041

<h2>4. Prediction for new data</h2>

Read the data to predict

In [18]:
new.data = read.csv(file=sprintf('%s/%s', folder_name, file_name))

Search for the best model in crossvalidation and use it to score the incoming data

In [19]:
best.model = which.max(r$measures.test$acc)

In [20]:
pred = predict(r$models[[best.model]], newdata = new.data)

Prediction Result

In [21]:
pred

Prediction: 1000 observations
predict.type: prob
threshold: 1=0.50,2=0.50
time: 0.01
  truth     prob.1    prob.2 response
1     1 0.73684211 0.2631579        1
2     2 0.07692308 0.9230769        2
3     1 0.86585366 0.1341463        1
4     1 0.33663366 0.6633663        2
5     2 0.33663366 0.6633663        2
6     1 0.86585366 0.1341463        1
... (1000 rows, 4 cols)


Cast result to data.frame to access the prediction

In [22]:
head(as.data.frame(pred))

truth,prob.1,prob.2,response
1,0.73684211,0.2631579,1
2,0.07692308,0.9230769,2
1,0.86585366,0.1341463,1
1,0.33663366,0.6633663,2
2,0.33663366,0.6633663,2
1,0.86585366,0.1341463,1
